In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
import numpy as np

from sklearn.model_selection import train_test_split
import PIL
import torchvision
from PIL import Image
from torchvision.transforms import v2

import os
import shutil
import random

import torchvision.transforms.functional as TF
from PIL import Image
from torchvision.transforms import CenterCrop, Compose, Pad, Resize, ToTensor

In [ ]:
folder1 = "/kaggle/input/acne-dataset/Acne"
folder2 = "/kaggle/input/oily-dry-and-normal-skin-types-dataset/Oily-Dry-Skin-Types/train/normal"

output_folder_normal = (
    "combined_dataset_acne_final/normal"
)
output_folder_acne = "combined_dataset_acne_final/acne"

# Создаём общую папку (если её нет)
os.makedirs(output_folder_normal, exist_ok=True)
os.makedirs(output_folder_acne, exist_ok=True)

for filename in os.listdir(folder1):
    src = os.path.join(folder1, filename)
    dst = os.path.join(output_folder_acne, filename)
    if os.path.isfile(src): 
        shutil.copy(src, dst)

for filename in os.listdir(folder2):
    src = os.path.join(folder2, filename)
    dst = os.path.join(output_folder_normal, filename)
    if os.path.isfile(src): 
        shutil.copy(src, dst)

In [ ]:
def resize_with_padding(image, target_size=(214, 214)):
    original_size = image.size
    ratio = min(target_size[0] / original_size[0], target_size[1] / original_size[1])
    new_size = (int(original_size[0] * ratio), int(original_size[1] * ratio))
    image = image.resize(new_size, Image.Resampling.LANCZOS)

    new_img = Image.new("RGB", target_size, (0, 0, 0))  
    paste_position = (
        (target_size[0] - new_size[0]) // 2,
        (target_size[1] - new_size[1]) // 2,
    )
    new_img.paste(image, paste_position)
    return new_img

In [9]:
from os import listdir
from os.path import isfile, join


from tqdm import tqdm

output_folder = "/kaggle/input/acne-data/Acne_data"

images_source = []
s = 0
for target, class_name in enumerate(["acne", "not_acne"]):
    if class_name == "acne":
        class_folder = f"{output_folder}/{class_name}"
        for image_name in tqdm(listdir(f"{output_folder}/{class_name}"), ncols=80):
            image_path = f"{class_folder}/{image_name}"        
            if not isfile(image_path):
                continue
            if not image_name.lower().endswith((".jpg", ".jpeg", ".png", ".bmp", ".gif", "JPG")):
                continue

            try:
                with Image.open(image_path) as img:
                    img = resize_with_padding(img, (214, 214))
                    images_source.append((img, target))
            except Exception as e:
                print(f"Ошибка при открытии {image_path}: {e}")
    else:
        class_folder = f"{output_folder}/{class_name}"
        for image_name in tqdm(listdir(f"{output_folder}/{class_name}"), ncols=80):
            if s < 550:
                image_path = f"{class_folder}/{image_name}"        
                if not isfile(image_path):
                    continue
                if not image_name.lower().endswith((".jpg", ".jpeg", ".png", ".bmp", ".gif")):
                    continue

                try:
                    with Image.open(image_path) as img:
                        img = resize_with_padding(img, (214, 214))
                        images_source.append((img, target))
                        s += 1
                except Exception as e:
                    print(f"Ошибка при открытии {image_path}: {e}")

print("\nВсего картинок:", len(images_source))

100%|██████████████████████████████████████| 1162/1162 [00:05<00:00, 219.82it/s]



Всего картинок: 1026


In [7]:
import zipfile

def zip_folder(folder_path, output_zip_path):
    with zipfile.ZipFile(output_zip_path, "w", zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                abs_path = os.path.join(root, file)
                rel_path = os.path.relpath(abs_path, folder_path)
                zipf.write(abs_path, rel_path)

zip_folder("combined_dataset_acne_final", "combined_dataset_acne_final.zip")


In [13]:
from random import shuffle

shuffle(images_source)
train_images_source = images_source[:800]
test_images_source = images_source[801:]


In [22]:
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms


class TnJDataset(Dataset):
    def __init__(self, source, is_train=True):
        self.is_train = is_train
        self.test_transform = transforms.Compose(
            [
                transforms.ToTensor(),
                transforms.Normalize(
                    mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
                ),
            ]
        )
        self.train_transform = transforms.Compose(
            [
                transforms.RandomHorizontalFlip(p=0.5),
                transforms.RandomRotation(degrees=10),
                transforms.GaussianBlur(kernel_size=(5, 5), sigma=(0.1, 1.5)),
                transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
                transforms.RandomAffine(degrees=10, translate=(0.05, 0.05)),
                transforms.RandomResizedCrop(214, scale=(0.9, 1.0)),
                transforms.ToTensor(),
                transforms.Normalize(
                    mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
                ),
            ]
        )
        data, target = list(zip(*source))
        self.data = data
        self.target = torch.tensor(target)

    def __getitem__(self, index):
        image, label = self.data[index], self.target[index]
        if self.is_train:
            image = self.train_transform(image)
        else:
            image = self.test_transform(image)
        return image, label

    def __len__(self):
        return len(self.data)

In [15]:
dataset_train = TnJDataset(train_images_source, is_train=True)
dataset_test = TnJDataset(test_images_source, is_train=False)

dataloader_train = DataLoader(
    dataset=dataset_train,
    batch_size=128,
    shuffle=True,
)
dataloader_test = DataLoader(
    dataset=dataset_test,
    batch_size=512,
    shuffle=False,
)


In [16]:
import torch.optim as optim
from torchvision import datasets, models, transforms

accuracy_arr = []


def train_model(train_loader, test_loader, n_epoch=16):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = models.efficientnet_b0(pretrained=True)

    for name, param in model.named_parameters():
        if "layer4" in name or "fc" in name:
            param.requires_grad = True
        else:
            param.requires_grad = False


    model.classifier[1] = nn.Linear(model.classifier[1].in_features, 2)

    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)


    for epoch in range(n_epoch):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for X, target in tqdm(train_loader, ncols=80):
            X, target = X.to(device), target.to(device)
            optimizer.zero_grad()

            logits = model(X)
            loss = criterion(logits, target)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * X.size(0)
            _, preds = torch.max(logits, 1)
            correct += torch.sum(preds == target).item()
            total += target.size(0)

        epoch_loss = running_loss / total
        epoch_acc = correct / total
        accuracy_arr.append(epoch_acc)
        print(
            f"Epoch {epoch+1}/{n_epoch} - Loss: {epoch_loss:.4f} - Acc: {epoch_acc:.4f}"
        )
        scheduler.step()

    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for X, target in tqdm(test_loader, ncols=80):
            X, target = X.to(device), target.to(device)
            logits = model(X)
            _, preds = torch.max(logits, 1)
            correct += torch.sum(preds == target).item()
            total += target.size(0)

    val_acc = correct / total
    print(f"Validation Accuracy: {val_acc:.4f}")
    return model


In [17]:
resnet_model = train_model(dataloader_train, dataloader_test)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 125MB/s] 
100%|█████████████████████████████████████████████| 7/7 [00:11<00:00,  1.71s/it]


Epoch 1/16 - Loss: 0.6930 - Acc: 0.5400


100%|█████████████████████████████████████████████| 7/7 [00:10<00:00,  1.52s/it]


Epoch 2/16 - Loss: 0.6579 - Acc: 0.6338


100%|█████████████████████████████████████████████| 7/7 [00:10<00:00,  1.52s/it]


...

Epoch 15/16 - Loss: 0.3879 - Acc: 0.8413


100%|█████████████████████████████████████████████| 7/7 [00:10<00:00,  1.54s/it]


Epoch 16/16 - Loss: 0.3827 - Acc: 0.8512


100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  2.01it/s]

Validation Accuracy: 0.8400


In [18]:
from PIL import Image
from torchvision import transforms

def prediction_final(model1, image_arr):
    transform = transforms.Compose(
        [
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ]
    )
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    acne_arr = []
    for i in image_arr:
        img = Image.open(i).convert("RGB")
        img = resize_with_padding(img, (214, 214))
        img_tensor = transform(img).unsqueeze(0)

        model1.eval()
        model1 = model1.to(device)

        img_tensor = img_tensor.to(device)

        with torch.no_grad():
            logits = model1(img_tensor)
            probs = torch.softmax(logits, dim=1)
            print(probs)
            predicted_class = torch.argmax(probs, dim=1).item()
            if abs(probs[0][0] - probs[0][1]) < 0.15:
                predicted_class = 2
            elif 0.15 < abs(probs[0][0] - probs[0][1]) < 0.3:
                predicted_class = 0

        class_names = ["Acne", "No Acne", "Hesitation"]
        acne_arr.append(class_names[predicted_class])
        
    if acne_arr.count("Acne") >= 1:
        return "Acne"
    if acne_arr.count("Hesitation") >= 1:
        return "Hesitation"
    return "No Acne"

In [20]:
torch.save(resnet_model, "/kaggle/working/full_resnet_model.pth")



In [ ]:
# print(prediction_final(resnet_model, ["testskin/masha.jpg", "testskin/masha1.jpg", "testskin/masha2.jpg"]))
# print(prediction_final(resnet_model, ["testskin/natasha.jpg", "testskin/natasha1.jpg", "testskin/natasha2.jpg"]))
# print(prediction_final(resnet_model, ["testskin/1.jpg", "testskin/2.jpg", "testskin/3.jpg"]))
# print(prediction_final(resnet_model, ["testskin/4.jpg", "testskin/5.jpg", "testskin/6.jpg"]))
# print(prediction_final(resnet_model, ["testskin/yarik.jpg", "testskin/yarik1.jpg", "testskin/yarik2.jpg"]))